# Import

In [1]:
import json
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta

# function to download stock data

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Write heading of the columns
        f.write('Date,Open,High,Low,Close,Volume\n')
        # load share data for the time period needed
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Choose the right columns
        stock_data.to_csv(f, header=False)  # Write the data to the file

# Define the shares

In [3]:
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']

# Read start and end for the sequence, which will be predicted from Excel

In [4]:
excel_file_path = 'settings/actualMonth_startEnd.xlsx'

# Path to file, in which the index is stored
index_file_path = 'settings/actualMonthIndex.txt'

# Read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0
print(current_row_index)
# read excel file
df = pd.read_excel(excel_file_path)

# Check, if the index is out of bounds
if current_row_index >= len(df):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract the data from the current row
    start_date = df.loc[current_row_index, 'start_date']
    end_date = df.loc[current_row_index, 'end_date']

    # Output of the current data
    print(f'Startdatum: {start_date}, Enddatum: {end_date}')

    # Updating the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

14
Startdatum: 2024-03-01, Enddatum: 2024-03-31


# Read close values at the begin of the time period

In [5]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = start_date_obj - timedelta(days=1)

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2024-02-25 00:00:00
2024-02-29 00:00:00


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)

print(combined_data_begin)

     Symbol  Close_Value
0    ALV.DE   248.550003
1    DBK.DE    12.396000
2   VOW3.DE   125.879997
3    BMW.DE   109.160004
4    ADS.DE   189.839996
5    BEI.DE   137.699997
6    DTE.SG    21.959999
7    SAP.DE   172.119995
8   1COV.DE    49.430000
9    BAS.DE    46.970001
10  EOAN.DE    11.735000
11   RWE.DE    30.790001


# read predictions

In [7]:
# Define the name of the file
json_file = 'settings/predictions_dict.json'

# read array from the json file
with open(json_file, 'r') as f:
    predictions_dict = json.load(f)
    
# Convert predictions_dict to DataFrame
predictions_df = pd.DataFrame(predictions_dict)
print(predictions_df)

                              ALV.DE     DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead  143.810287  14.912164  86.344261  86.055275   

                              ADS.DE      BEI.DE     DTE.SG     SAP.DE  \
prediction_20_days_ahead  105.564362  102.767632  20.717569  82.988327   

                            1COV.DE     BAS.DE    EOAN.DE    RWE.DE  
prediction_20_days_ahead  48.966732  46.389107  13.493193  32.02832  


# Calculate return in percentage

In [8]:
# Für jedes Symbol in predictions_df den entsprechenden Close_Value aus combined_data_begin abziehen
for symbol in predictions_df.columns:
    if symbol in combined_data_begin['Symbol'].values:
        close_value = combined_data_begin.loc[combined_data_begin['Symbol'] == symbol, 'Close_Value'].values[0]
        predictions_df[symbol] = ((predictions_df[symbol] - close_value) / close_value) * 100

print(predictions_df)   

                             ALV.DE     DBK.DE    VOW3.DE     BMW.DE  \
prediction_20_days_ahead -42.140299  20.298192 -31.407481 -21.165929   

                             ADS.DE     BEI.DE    DTE.SG    SAP.DE   1COV.DE  \
prediction_20_days_ahead -44.392982 -25.368458 -5.657695 -51.78461 -0.937221   

                            BAS.DE    EOAN.DE    RWE.DE  
prediction_20_days_ahead -1.236735  14.982472  4.021823  


# read profit

In [9]:
json_file_path = 'settings/profit.json'

try:
    # read JSON file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
        profit = data['profit']
except FileNotFoundError:
    # If the file can not be found, set the value of new_portfolio_value to 0
    profit = 0

print(f'Profit: {profit}')

Profit: -2.0761984664113697


# load share value from 2019

In [10]:
start_date = start_date
end_date = end_date
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

Startdatum: 2024-03-01, Enddatum: 2024-03-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])
# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

277.79998779296875
Liste der letzten Close-Werte:
ALV.DE: 277.79998779296875
DBK.DE: 14.58199977874756
VOW3.DE: 122.83999633789062
BMW.DE: 106.95999908447266
ADS.DE: 207.0
BEI.DE: 134.9499969482422
DTE.SG: 22.434999465942383
SAP.DE: 180.4600067138672
1COV.DE: 50.68000030517578
BAS.DE: 52.93000030517578
EOAN.DE: 12.885000228881836
RWE.DE: 31.459999084472656


# Historic data from 2018 for the covarianz matrix

In [12]:
excel_file_path = 'settings/lastSequenceForPrediction.xlsx'

# path to the data, in which the index is stored
index_file_path = 'settings/lastSequenceForPrediction.txt'

# read the current index from the index file
try:
    with open(index_file_path, 'r') as index_file:
        current_row_index = int(index_file.read().strip())
except FileNotFoundError:
    current_row_index = 0

# read excel file
df_lastSeq = pd.read_excel(excel_file_path)

# Check, if index is out of bounds
if current_row_index >= len(df_lastSeq):
    print("Es gibt keine weiteren Zeilen in der Excel-Tabelle.")
else:
    # extract data from the current row
    start_date_lastSeq = df_lastSeq.loc[current_row_index, 'start_date']
    end_date_lastSeq = df_lastSeq.loc[current_row_index, 'end_date']
    
    # Output of the current data 
    print(f'Startdatum: {start_date_lastSeq}, Enddatum: {end_date_lastSeq}')

    # Update the index in the index file for the next call
    with open(index_file_path, 'w') as index_file:
        index_file.write(str(current_row_index + 1))

Startdatum: 2023-10-01, Enddatum: 2024-02-28


In [13]:
# Definition of the time period
start_date2018 = '2023-01-01'
end_date2018 = end_date_lastSeq
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


2024-02-28


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file 

In [14]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)
    
# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE  DBK.DE     VOW3.DE      BMW.DE      ADS.DE  \
2023-01-02  203.050003  10.942  120.040001   85.800003  127.699997   
2023-01-03  205.199997  11.112  122.059998   85.830002  131.880005   
2023-01-04  211.500000  11.698  125.879997   87.879997  138.380005   
2023-01-05  209.550003  11.494  127.120003   88.949997  138.539993   
2023-01-06  211.800003  11.596  128.160004   89.529999  140.679993   
...                ...     ...         ...         ...         ...   
2024-02-21  250.699997  11.994  119.580002  104.300003  181.440002   
2024-02-22  255.100006  12.252  122.400002  105.080002  187.820007   
2024-02-23  246.500000  12.390  124.000000  106.720001  188.779999   
2024-02-26  245.649994  12.302  123.019997  107.480003  188.199997   
2024-02-27  248.000000  12.384  124.080002  107.680000  188.160004   

                BEI.DE     DTE.SG      SAP.DE    1COV.DE     BAS.DE  EOAN.DE  \
2023-01-02  107.150002  18.910000   97.419998  38.259998  47.985001    9.468   

# Mean variance optimization

In [15]:
def mean_variance_optimization(expected_returns, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # start weighs
    bounds = [(0, 1)] * n  # weigh border (0-100% for each share)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x


# Portfilio allocation

In [16]:
def allocate_portfolio(predictions, historical_data, initial_capital):
    # Calculate expected returns based on the predictions
    expected_returns = predictions.mean()
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation


# Calculate portfolio

In [17]:
initial_capital = 1000 + profit # 1000€ start capital
print(initial_capital)
# Calculate the portfolio allocation
portfolio_allocation = allocate_portfolio(predictions_df, combined_historical2018_data, initial_capital)
# print portfolio allocation
print(portfolio_allocation)

997.9238015335886
[0.00000000e+00 3.79188303e+02 5.04999224e+00 2.10468137e-05
 0.00000000e+00 0.00000000e+00 2.18077679e-05 0.00000000e+00
 1.37589594e-05 2.87537453e-05 5.77318450e+02 3.63671735e+01]


# Simulation for one month

In [18]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

0.0
0.0
0.0
-----------------------------
30.589569699724414
379.1883031972251
379.1883031972251
-----------------------------
0.04011751148619983
5.049992235696798
384.23829543292186
-----------------------------
1.9280700804914174e-07
2.1046813704724665e-05
384.23831647973554
-----------------------------
0.0
0.0
384.23831647973554
-----------------------------
0.0
0.0
384.23831647973554
-----------------------------
9.930677961166368e-07
2.1807767893540624e-05
384.23833828750344
-----------------------------
0.0
0.0
384.23833828750344
-----------------------------
2.783524037583593e-07
1.375895940272211e-05
384.23835204646286
-----------------------------
6.121725469075133e-07
2.875374527552684e-05
384.2383808002081
-----------------------------
49.196290339661026
577.3184502457162
961.5568310459244
-----------------------------
1.1811358360276671
36.367173472654024
997.9240045185784
-----------------------------
Gesamtwert des Portfolios am Ende des ersten Monats von 2019: 997.9240

In [19]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

277.79998779296875
0.0
0.0
----------------------------
14.58199977874756
30.589569699724414
446.0570985933645
----------------------------
122.83999633789062
0.04011751148619983
450.9851335574146
----------------------------
106.95999908447266
1.9280700804914174e-07
450.985154180052
----------------------------
207.0
0.0
450.985154180052
----------------------------
134.9499969482422
0.0
450.985154180052
----------------------------
22.434999465942383
9.930677961166368e-07
450.9851764595275
----------------------------
180.4600067138672
0.0
450.9851764595275
----------------------------
50.68000030517578
2.783524037583593e-07
450.98519056642743
----------------------------
52.93000030517578
6.121725469075133e-07
450.9852229687205
----------------------------
12.885000228881836
49.196290339661026
1084.87943525539
----------------------------
31.459999084472656
1.1811358360276671
1122.0379675754582
----------------------------
Wert des Portfolios:
1122.0379675754582


In [20]:
# just the profit, to store it
profit = new_portfolio_value - 1000

# path to JSON file
json_file_path = 'settings/profit.json'

# Create Dictionary with the value of new_portfolio_value
data = {'profit': profit}

# Write JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)

print(f'Wert von profit wurde erfolgreich in {json_file_path} gespeichert.')

Wert von profit wurde erfolgreich in settings/profit.json gespeichert.
